In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, LeakyReLU, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import time


In [3]:
class GAN():
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.batch_size = 64
        self.n_discriminator = 5
        self.lambda_gp = 10
        self.class_label = 1  # CIFAR-10 class for automobiles

        self.generator = self.build_generator()
        self.discriminator = self.build_discriminator()

        self.g_optimizer = Adam(learning_rate=1e-4, beta_1=0.5, beta_2=0.9)
        self.d_optimizer = Adam(learning_rate=1e-4, beta_1=0.5, beta_2=0.9)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(4 * 4 * 512, input_dim=self.latent_dim))
        model.add(Reshape((4, 4, 512)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Conv2D(self.channels, kernel_size=3, padding="same", activation="tanh"))

        return model

    def build_discriminator(self):
        model = Sequential()

        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same", input_shape=self.img_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(1))

        return model

    def gradient_penalty(self, real, fake):
        batch_size = tf.shape(real)[0]
        epsilon = tf.random.uniform([batch_size, 1, 1, 1], 0.0, 1.0)
        interpolated = epsilon * real + (1 - epsilon) * fake
        with tf.GradientTape() as tape:
            tape.watch(interpolated)
            pred = self.discriminator(interpolated, training=True)
        grads = tape.gradient(pred, [interpolated])[0]
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1,2,3]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp

    def wasserstein_loss(self, y_true, y_pred):
        return tf.reduce_mean(y_true * y_pred)

    @tf.function
    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        for _ in range(self.n_discriminator):
            noise = tf.random.normal([batch_size, self.latent_dim])
            with tf.GradientTape() as tape:
                fake_images = self.generator(noise, training=True)
                real_output = self.discriminator(real_images, training=True)
                fake_output = self.discriminator(fake_images, training=True)
                gp = self.gradient_penalty(real_images, fake_images)
                d_loss = tf.reduce_mean(fake_output) - tf.reduce_mean(real_output) + self.lambda_gp * gp
            gradients = tape.gradient(d_loss, self.discriminator.trainable_variables)
            self.d_optimizer.apply_gradients(zip(gradients, self.discriminator.trainable_variables))

        noise = tf.random.normal([batch_size, self.latent_dim])
        with tf.GradientTape() as tape:
            fake_images = self.generator(noise, training=True)
            fake_output = self.discriminator(fake_images, training=True)
            g_loss = -tf.reduce_mean(fake_output)
        gradients = tape.gradient(g_loss, self.generator.trainable_variables)
        self.g_optimizer.apply_gradients(zip(gradients, self.generator.trainable_variables))

        # Calculate accuracy
        real_accuracy = tf.reduce_mean(tf.cast(real_output < 0, tf.float32))
        fake_accuracy = tf.reduce_mean(tf.cast(fake_output >= 0, tf.float32))
        accuracy = (real_accuracy + fake_accuracy) / 2

        return d_loss, g_loss, accuracy

    def train(self, epochs, save_interval=100):
        # Load the dataset
        (x_train, y_train), (_, _) = cifar10.load_data()
        x_train = x_train[y_train.flatten() == self.class_label]
        x_train = x_train.astype('float32')
        x_train = (x_train - 127.5) / 127.5  # Scale to [-1, 1]
        dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(10000).batch(self.batch_size)

        for epoch in range(epochs):
            start = time.time()
            d_loss_total = 0
            g_loss_total = 0
            accuracy_total = 0
            num_batches = 0
            for real_images in dataset:
                d_loss, g_loss, accuracy = self.train_step(real_images)
                d_loss_total += d_loss
                g_loss_total += g_loss
                accuracy_total += accuracy
                num_batches += 1

            d_loss_avg = d_loss_total / num_batches
            g_loss_avg = g_loss_total / num_batches
            accuracy_avg = accuracy_total / num_batches
            print(f"Epoch: {epoch}, Discriminator Loss: {d_loss_avg:.4f}, Generator Loss: {g_loss_avg:.4f}, Accuracy: {accuracy_avg:.4f}, Time: {time.time()-start:.2f}")

            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = tf.random.normal((r * c, self.latent_dim))
        gen_imgs = self.generator(noise, training=False)

        gen_imgs = tf.clip_by_value(0.5 * gen_imgs + 0.5, 0.0, 1.0)

        fig, axs = plt.subplots(r, c, figsize=(5, 5))
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt].numpy())
                axs[i, j].axis('off')
                cnt += 1
        os.makedirs("automobile_images", exist_ok=True)
        fig.savefig(f"automobile_images/epoch_{epoch}.png")
        plt.close(fig)



In [ ]:
if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=1000, save_interval=50)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch: 0, Discriminator Loss: -8.7033, Generator Loss: 10.3025, Accuracy: 0.2276, Time: 41.23
Epoch: 1, Discriminator Loss: -6.1160, Generator Loss: 9.7688, Accuracy: 0.3956, Time: 3.82
Epoch: 2, Discriminator Loss: -5.6847, Generator Loss: 10.0960, Accuracy: 0.4385, Time: 3.86
Epoch: 3, Discriminator Loss: -3.7260, Generator Loss: 6.2715, Accuracy: 0.3830, Time: 3.83
Epoch: 4, Discriminator Loss: -3.0740, Generator Loss: 3.3548, Accuracy: 0.2770, Time: 3.84
Epoch: 5, Discriminator Loss: -2.9439, Generator Loss: 1.4293, Accuracy: 0.2543, Time: 3.87
Epoch: 6, Discriminator Loss: -2.6791, Generator Loss: 0.8037, Accuracy: 0.2792, Time: 3.78
Epoch: 7, Discriminator Loss: -2.4719, Generator Loss: 0.2257, Accuracy: 0.2924, Time: 3.81
Epoch: 8, Discriminator Loss: -2.2464, Generator Loss: -0.2253, Accuracy: 0.3355, Time: 3.92
Epoch: 9, Discriminator Loss: -2.1138, Generator Loss: -0.1688, Accuracy: 0.3315, Time: 3.83
Epoch: 10, Discriminat